In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

/kaggle/input/tunes-data/data.txt


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
import warnings
warnings.filterwarnings('ignore')
def split_text_into_characters(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
        characters = list(text)  # Splitting the text into individual characters

    return characters
path='/kaggle/input/tunes-data/data.txt'
tokens=split_text_into_characters(path)
print(len(tokens))
vocab=pd.Series(tokens).unique()
print(vocab.shape)

452328
(93,)


In [3]:
vocab

array(['\n', 'X', ':', '1', 'T', 'A', ' ', 'a', 'n', 'd', "'", 's', 'W',
       'l', 't', 'z', '%', 'N', 'o', 'i', 'g', 'h', 'm', 'M', 'u', 'c',
       'D', 'b', 'e', 'S', 'k', 'P', '3', '/', '4', 'L', 'K', 'G', '|',
       '"', '2', 'B', '[', ']', 'F', '=', 'C', 'f', 'r', 'y', 'v', '(',
       'E', '\\', '^', '#', '6', '8', 'p', 'H', '5', 'R', '7', '_', '9',
       '0', 'J', ')', 'I', 'j', '-', '~', 'w', 'O', '&', ',', '.', 'Y',
       '?', 'V', 'x', '+', 'q', '!', '>', 'Q', 'U', '*', '@', '<', '{',
       '}', 'Z'], dtype=object)

In [4]:
char_to_idx={}
for i in range(vocab.shape[0]):
    char_to_idx[vocab[i]]=i
tokens=pd.Series(tokens).map(char_to_idx)
tokens

0          0
1          1
2          2
3          3
4          0
          ..
452323    38
452324    38
452325     0
452326     0
452327     0
Length: 452328, dtype: int64

In [6]:
def create_batches(data,batch_size,seq_len,vocab):
    n=len(data)
    batch_count=n//(batch_size*seq_len)
    X=[]
    for i in range(batch_count):
        x_batch=[]
        idx=i*seq_len
        for row in range(batch_size):
            x_temp=[]
            for j in range(seq_len):
                x_temp.append(data[idx+j])
            x_batch.append(x_temp)
            idx+=(seq_len*batch_count)
#             print(i,row,idx)
        X.append(x_batch)
    
    return X

seq_len=256
batch_size=128
real_data=create_batches(tokens,len(tokens)//seq_len,seq_len,vocab)
real_data=np.array(real_data)
real_data=real_data.reshape((1766,256))
real_data.shape

(1766, 256)

In [65]:
# import numpy as np
# from keras.models import Sequential
# from keras.layers import Dense, LSTM , Input

# # Generator architecture
# num_chars=vocab.shape[0]
# generator = Sequential()
# generator.add(LSTM(128, input_shape=(seq_length, num_chars)))
# generator.add(Dense(num_chars, activation='softmax'))
# generator.compile(loss='categorical_crossentropy', optimizer='adam')

# # Discriminator architecture
# discriminator = Sequential()
# discriminator.add(LSTM(128, input_shape=(seq_length, num_chars)))
# discriminator.add(Dense(1, activation='sigmoid'))
# discriminator.compile(loss='binary_crossentropy', optimizer='adam')

# # Combined GAN model
# discriminator.trainable = False
# gan_input = Input(shape=(seq_length, num_chars))
# generated_sequence = generator(gan_input)
# gan_output = discriminator(generated_sequence)
# gan = Model(gan_input, gan_output)
# gan.compile(loss='binary_crossentropy', optimizer='adam')

# # Training loop
# for epoch in range(num_epochs):
#     # Generate a batch of real sequences
#     real_sequences = real_data

#     # Generate a batch of random noise vectors
#     noise = np.random.normal(0, 1, size=(batch_size, seq_length, num_chars))

#     # Generate a batch of fake sequences using the generator
#     generated_sequences = generator.predict(noise)

#     # Train the discriminator on real and fake sequences
#     discriminator_loss_real = discriminator.train_on_batch(real_sequences, np.ones((batch_size, 1)))
#     discriminator_loss_fake = discriminator.train_on_batch(generated_sequences, np.zeros((batch_size, 1)))
#     discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)

#     # Train the generator to fool the discriminator
#     gan_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

#     # Print the losses for monitoring
#     print(f"Epoch: {epoch+1}/{num_epochs}, Discriminator Loss: {discriminator_loss}, GAN Loss: {gan_loss}")


In [66]:
# import tensorflow as tf
# from tensorflow.keras import layers

# latent_dim = 100  # Size of the random noise input vector
# sequence_length = seq_len  # Length of input sequence

# # Generator architecture
# generator = tf.keras.Sequential()
# generator.add(layers.Reshape((1, latent_dim), input_shape=(latent_dim,)))
# generator.add(layers.LSTM(256, input_shape=(latent_dim,), return_sequences=True))
# generator.add(layers.BatchNormalization())
# generator.add(layers.LSTM(512, return_sequences=True))
# generator.add(layers.BatchNormalization())
# generator.add(layers.TimeDistributed(layers.Dense(128, activation='relu')))
# generator.add(layers.BatchNormalization())
# generator.add(layers.TimeDistributed(layers.Dense(1, activation='sigmoid')))
# generator.build(input_shape=(None, latent_dim))
# generator.summary()
# # Discriminator architecture
# discriminator = tf.keras.Sequential()
# discriminator.add(layers.LSTM(512, input_shape=(sequence_length, 1), return_sequences=True))
# discriminator.add(layers.BatchNormalization())
# discriminator.add(layers.LSTM(256, return_sequences=True))
# discriminator.add(layers.BatchNormalization())
# discriminator.add(layers.TimeDistributed(layers.Dense(128, activation='relu')))
# discriminator.add(layers.BatchNormalization())
# discriminator.add(layers.TimeDistributed(layers.Dense(1, activation='sigmoid')))
# discriminator.build(input_shape=(None, sequence_length, 1))
# discriminator.summary()
# # Combined GAN model
# gan = tf.keras.Sequential([generator, discriminator])
# gan.build(input_shape=(None, latent_dim))

# # Rest of the code (compilation, training loop, etc.)
# gan.summary()

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_5 (Reshape)         (None, 1, 100)            0         
                                                                 
 lstm_44 (LSTM)              (None, 1, 256)            365568    
                                                                 
 batch_normalization_27 (Bat  (None, 1, 256)           1024      
 chNormalization)                                                
                                                                 
 lstm_45 (LSTM)              (None, 1, 512)            1574912   
                                                                 
 batch_normalization_28 (Bat  (None, 1, 512)           2048      
 chNormalization)                                                
                                                                 
 time_distributed_18 (TimeDi  (None, 1, 128)         

In [19]:
def create_g():
    model=keras.models.Sequential()
    model.add(keras.layers.Dense(64))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(128))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(vocab.shape[0],activation='tanh'))
#     model.summary()
    model.compile(optimizer='adam',loss=keras.losses.binary_crossentropy)
    return model

In [20]:
def create_d():
    model=keras.models.Sequential()
    model.add(keras.layers.Embedding(input_dim=vocab.shape[0],output_dim=512,input_length=seq_len))
    for i in range(3):
        model.add(keras.layers.LSTM(128,return_sequences=(i!=2)))
    model.add(keras.layers.Dense(1,activation='sigmoid'))
#     model.summary()
    model.compile(optimizer='adam',loss=keras.losses.binary_crossentropy)
    return model

In [21]:
from tensorflow.keras.layers import Layer

class OneHotToIndex(Layer):
    def __init__(self):
        super(OneHotToIndex, self).__init__()

    def call(self, inputs):
        return tf.argmax(inputs, axis=-1)

In [24]:
def create_gan(g,d):
    d.trainable=False
    inp=keras.layers.Input(shape=seq_len)
    x=g(inp)
#     x=OneHotToIndex()(x)
    out=d(x)
    model=keras.Model(inputs=inp,outputs=out)
    model.summary()
    model.compile(optimizer=keras.optimizers.Adam(0.0002,0.5),loss=keras.losses.binary_crossentropy)
    return model

In [25]:
create_gan(create_g(),create_d())

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 256)]             0         
                                                                 
 sequential_8 (Sequential)   (None, 93)                36765     
                                                                 
 sequential_9 (Sequential)   (None, 1)                 639105    
                                                                 
Total params: 675,870
Trainable params: 36,765
Non-trainable params: 639,105
_________________________________________________________________


In [16]:
def view_music(g):
    noise=np.random.normal(0,vocab.shape[0],size=(batch_size,seq_len))
    gen_data=g.predict(noise)
    data=OhetoInd(gen_data)
    notes=3
    for i in range(notes):
        music=[vocab[j] for j in data[i]]
        output='a'
        for v in music:
            output+=v
        print(output)
        print('--------------------------')

In [14]:
def OhetoInd(data):
    music=[]
    for i in range(data.shape[0]):
        temp=[]
        for j in range(data.shape[1]):
            temp.append(np.argmax(data[i][j]))
        music.append(temp)
    return music

In [26]:
epochs=100
batch_size=64
data=real_data
g=create_g()
d=create_d()
gan=create_gan(g,d)
    
for e in range(1,epochs+1):
    noise=np.random.randint(0,vocab.shape[0],size=(batch_size,seq_len))
    gen_data=g.predict(noise)
    gen_data
    gen_data=OhetoInd(gen_data)

    if e==1 or e%100==0:
        view_music(g)
training(100,64,real_data)

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 256)]             0         
                                                                 
 sequential_10 (Sequential)  (None, 93)                36765     
                                                                 
 sequential_11 (Sequential)  (None, 1)                 639105    
                                                                 
Total params: 675,870
Trainable params: 36,765
Non-trainable params: 639,105
_________________________________________________________________
2/2 [==============================] - 0s 3ms/step
a





























































































--------------------------
a





























































































--------------------------
a

















KeyboardInterrupt: 

In [75]:
real_data=data[np.random.randint(0,data.shape[0],size=batch_size)]
train=np.concatenate([real_data,gen_data])
train,train.shape

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 256 and the array at index 1 has size 93

In [76]:
labels=np.zeros(2*batch_size)
labels[:batch_size]=0.9
d.trainable=True
d.fit(train,labels)

NameError: name 'train' is not defined

In [77]:
noise=np.random.randint(0,vocab.shape[0],size=(batch_size,seq_len))
y=np.ones(batch_size)
d.trainable=False
gan.fit(noise,y)

ValueError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/opt/conda/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: Exception encountered when